# Plant Seddlings Classification

### 설명
* 목적: 잡초와 농작물 모종을 구별하기 --> 더 나은 농작물 생산량 & 환경 관리
* 12개 종에 속하는 약 960개의 고유 식물의 이미지를 포함하는 데이터 셋

### 필사
* https://www.kaggle.com/code/nikkonst/plant-seedlings-with-cnn-and-image-processing

## 목차
### 1. 데이터 얻기
### 2. 데이터 정리하기
### 3. 모델링
### 4. 모델 평가하기

## **1. 데이터 얻기 - 이미지와 레이블 읽기**

In [ ]:
import cv2 # 영상을 처리하기 위한 라이브러리
from glob import glob # 파일들의 리스트를 뽑을 때 사용, 파일의 경로명을 이용해서 다양한 활용 가능
import numpy as np
from matplotlib import pyplot as plt
import math
import pandas as pd

ScaleTo = 70  # px to scale
seed = 7  # fixing random

path = '../input/plant-seedlings-classification/train/*/*.png' # 위 디렉토리의 png 파일 주소
files = glob(path) # glob로 files라는 변수에 저장

trainImg = []
trainLabel = []
cnt = 1
num = len(files)

# Obtain images and resizing, obtain labels
for img in files:
    print(str(cnt) + "/" + str(num), end="\r")
    trainImg.append(cv2.resize(cv2.imread(img), (ScaleTo, ScaleTo)))  # Get image (with resizing)
    trainLabel.append(img.split('/')[-2])  # Get image label (folder name)
    cnt += 1

trainImg = np.asarray(trainImg)  # Train images set
trainLabel = pd.DataFrame(trainLabel)  # Train labels set

In [ ]:
# Show some example images
for i in range(12):
    plt.subplot(3, 4, i + 1) # subplot(row, column, index) : 여러 개의 그래프를 그리기
    plt.imshow(trainImg[i])

* 위 그림을 통해 배경이 있는 것을 확인 -> 정확도를 위해선 배경을 없애는 것이 좋음.

## **2.데이터 정리하기**

배경을 지우기 위해선 식물이 초록색인 점을 활용하면 된다.
<br>
mask를 만들고, 초록색 이외에 부분을 날리면 된다.

### 2.1 초록 식물 마스킹하기
마스크를 만들기 위해선 배경을 지워야하는데, 이를 위해선 우린 RGB 이미지를 HSV로 변환해야한다.
<br>
HSV는 RGB 컬러 모델의 대안으로 RGB 컬러 공간보다 색 범위 표현이 쉽다.
<br>
이외에도 노이즈 제거를 위해 이미지를 블러처리한다.
<br>
#### 할 것
1. 노이즈 제거를 위한 블러 생성 with Gaussian
2. HSV 이미지로 변환
3. 녹색의 선택된 범위를 기반하여 마스크 생성.
4. 부울 마스크로 변환하여 본래 이미지에 적용

In [ ]:
clearTrainImg = [] # 정리한 이미지를 넣을 배열
examples = []; getEx = True
for img in trainImg:
    # Use gaussian blur
    # cv2.GaussianBlur(src, ksize(커널 크기), sigmaX, sigmaY, 기타 파라미터들...)
    blurImg = cv2.GaussianBlur(img, (5, 5), 0)
    
    # Convert to HSV image
    hsvImg = cv2.cvtColor(blurImg, cv2.COLOR_BGR2HSV)  
    
    # Create mask (parameters - green color range)    
    lower_green = (25, 40, 50)
    upper_green = (75, 255, 255)
   
    ## 특정 범위 안에 있는 행렬 원소 검출 - cv2.inRange(src, lowerb, upperb, dst)
    mask = cv2.inRange(hsvImg, lower_green, upper_green)
    
    ## getStructuringElement(shape, ksize, anchor): 이미지를 형태학적 관점에서 접근하는 기법
    ### shape: 구조화 요소 커널의 모양 
    ###     ==> cv2.MORPH_CROSS / cv2.MORPH_ELLIPSE / cv2.MORPH_RECT 
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    # Create bool mask
    bMask = mask > 0  
    
    # Apply the mask
    clear = np.zeros_like(img, np.uint8)  # Create empty image
    clear[bMask] = img[bMask]  # Apply boolean mask to the origin image
    
    clearTrainImg.append(clear)  # Append image without backgroung
    
    # Show examples
    if getEx:
        plt.subplot(2, 3, 1); plt.imshow(img)  # Show the original image
        plt.subplot(2, 3, 2); plt.imshow(blurImg)  # Blur image
        plt.subplot(2, 3, 3); plt.imshow(hsvImg)  # HSV image
        plt.subplot(2, 3, 4); plt.imshow(mask)  # Mask
        plt.subplot(2, 3, 5); plt.imshow(bMask)  # Boolean mask
        plt.subplot(2, 3, 6); plt.imshow(clear)  # Image without background
        getEx = False
        
clearTrainImg = np.asarray(clearTrainImg)

In [ ]:
# Show sample result
for i in range(12):
    plt.subplot(3, 4, i + 1)
    plt.imshow(clearTrainImg[i])

이렇게 배경을 제외한 사진들을 얻을 수 있었다.

### 2.2 Normalize input
RGB 컬러 공간은 [0...255] 까지다. CNN이 더 빠르기 위해선 [0...1] 로 변환해야한다.

In [ ]:
clearTrainImg = clearTrainImg / 255

### 2.3 Categories labels
12가지의 종류을 가진 라벨을 0 또는 1로만 이루어진 벡터로 값을 수정해야합니다.
<br>
For example 'Charlock' -> [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0].

In [ ]:
from keras.utils import np_utils
from sklearn import preprocessing
import matplotlib.pyplot as plt

# Encode labels and create classes
le = preprocessing.LabelEncoder()
le.fit(trainLabel[0])
print("Classes: " + str(le.classes_))
encodeTrainLabels = le.transform(trainLabel[0])

# Make labels categorical
clearTrainLabel = np_utils.to_categorical(encodeTrainLabels)
num_clases = clearTrainLabel.shape[1]
print("Number of classes: " + str(num_clases))

# Plot of label types numbers
trainLabel[0].value_counts().plot(kind='bar')

## **3. 모델링**

## 3.1 데이터셋 분리하기
train 데이터와 validation 데이터셋으로 나눈다. 10% 데이터가 validation 셋이 된다. 
<br>
현재 데이터 셋은 불균형하다. 부정확한 모델셋을 피하기 위해 stratify = clearTrainLabel를 쓴다.

In [ ]:
from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = train_test_split(clearTrainImg, clearTrainLabel, 
                                                test_size=0.1, random_state=seed, 
                                                stratify = clearTrainLabel)

### 3.2 데이터 생성기
과적합을 방지하기 위해 모델이 fitting하는 동안 이미지를 임의로 회전, 확대/축소, 이동 및 뒤집는 **이미지 생성기**를 만들어야 합니다.

* 임의 회전을 0도에서 180도로 설정합니다.
* 임의 줌을 0.1로 설정
* 임의 이동을 0.1로 설정합니다.
* 수평 및 수직 플립 설정

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        rotation_range=180,  # randomly rotate images in the range
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally
        height_shift_range=0.1,  # randomly shift images vertically 
        horizontal_flip=True,  # randomly flip images horizontally
        vertical_flip=True  # randomly flip images vertically
    )  
datagen.fit(trainX)

### 3.3 모델 생성하기
생성 모델을 위해선 Keras Sequential을 사용.
<br>
6개 컨볼루션 층
3개 fully-conncected 층
<br>
나는 여섯 개의 컨볼루션 레이어와 세 개의 완전히 연결된 레이어를 가진 모델을 만들었다. 처음 두 개의 컨볼루션 레이어는 64개의 필터를 가지고 있고, 다음 128개의 필터와 마지막 두 개의 레이어는 256개의 필터를 가지고 있다. 각 컨볼루션 레이어 쌍 이후 모델에는 최대 풀링 레이어가 있습니다. 또한 각 컨볼루션 레이어 쌍 이후 과적합을 줄이기 위해 드롭아웃 레이어(컨볼루션 레이어 간 10%, 완전히 연결된 레이어 간 50%)를 사용하고 각 레이어 간에는 배치 정규화 레이어를 사용한다.

결국 나는 분류를 위해 완전히 연결된 세 개의 레이어를 사용했다. 마지막 레이어에서 신경망은 12개 클래스 각각에 대한 확률 분포를 출력한다.

In [ ]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import BatchNormalization

numpy.random.seed(seed)  # Fix seed

model = Sequential()

model.add(Conv2D(filters=64, kernel_size=(5, 5), input_shape=(ScaleTo, ScaleTo, 3), activation='relu'))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.1))

model.add(Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.1))

model.add(Conv2D(filters=256, kernel_size=(5, 5), activation='relu'))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(filters=256, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(num_clases, activation='softmax'))

model.summary()

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### 3.4 Fit 모델
여기서 모델을 교육하겠습니다. 먼저, 몇 가지 콜백을 설정했습니다. 첫 번째 콜백은 모델 학습률을 감소시킨다. 높은 학습률 모델에서는 모델이 더 빨리 수렴되지만, 높은 학습률에서는 모델이 로컬 최소값으로 떨어질 수 있다. 그래서, 우리는 피팅하는 과정에서 학습률을 낮출 것입니다. 3세기 동안 정확도가 향상되지 않으면 학습률을 낮출 것입니다. 다른 두 가지 콜백은 모델의 최고 및 마지막 가중치를 절약합니다.

우리는 kaggle 커널에서 모델을 훈련시키지 않을 것이다. 왜냐하면 그것은 너무 긴 과정이기 때문이다. 그래서 나는 fitting으로 코드 라인을 코멘트한다.

In [ ]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger

# learning rate reduction
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.4, 
                                            min_lr=0.00001)

# checkpoints
filepath="drive/DataScience/PlantReco/weights.best_{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', 
                             verbose=1, save_best_only=True, mode='max')
filepath="drive/DataScience/PlantReco/weights.last_auto4.hdf5"
checkpoint_all = ModelCheckpoint(filepath, monitor='val_accuracy', 
                                 verbose=1, save_best_only=False, mode='max')

# all callbacks
callbacks_list = [checkpoint, learning_rate_reduction, checkpoint_all]

# fit model
hist = model.fit_generator(datagen.flow(trainX, trainY, batch_size=75), 
                            epochs=35, validation_data=(testX, testY), 
                            steps_per_epoch=trainX.shape[0], callbacks=callbacks_list)

## **4. 모델 평가하기**

### 4.1. 파일에서 모델 불러오기
여기서는 파일에서 최적의 모델의 가중치를 로드한다(앞서 훈련한 모델의 가중치가 있는 카글 데이터 세트를 사용했다). 또한 모델 정확도 평가에 적합한 모델을 Data.npz 교육 및 검증 데이터 세트에서 로드합니다

### 4.3 결과 얻기

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    
    fig = plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
predY = model.predict(testX)
predYClasses = np.argmax(predY, axis = 1) 
trueY = np.argmax(testY, axis = 1) 

# confusion matrix
confusionMTX = confusion_matrix(trueY, predYClasses) 

# plot the confusion matrix
plot_confusion_matrix(confusionMTX, classes = le.classes_) 